# Generator-level data with python-entsoe

This notebook answers whether ENTSO-E exposes individual generator data. The API does expose per-plant generation via document type A73, but only for units with at least 100 MW installed capacity and only where the TSO publishes it.

For Germany, try `DE_LU` first (bidding zone used by many datasets), then `DE`. Availability can vary by period and may be incomplete compared to national sources like Elexon.

In [4]:
import pandas as pd
from entsoe import Client, NoDataError

client = Client()  # reads ENTSOE_API_KEY from env

## Pick a small time window

Use a short window first to confirm availability. This notebook tries a few 24-hour windows in case some periods are missing.

In [5]:
date_windows = [
    ("2024-06-03", "2024-06-04"),
    ("2024-01-15", "2024-01-16"),
    ("2023-11-01", "2023-11-02"),
]
area_candidates = ["DE_LU", "DE"]

def fetch_per_plant(areas, windows):
    errors = []
    for start, end in windows:
        for area in areas:
            try:
                df = client.generation.per_plant(start, end, country=area)
                df["area"] = area
                df["start"] = start
                df["end"] = end
                return df, start, end, errors
            except NoDataError as exc:
                errors.append(f"{area} {start} to {end}: {exc}")
    return None, None, None, errors

## Fetch generator-level data

In [6]:
df, used_start, used_end, errors = fetch_per_plant(area_candidates, date_windows)
if df is None:
    print("No per-plant data found for the requested areas/windows.")
    print("Tried:")
    for msg in errors[:5]:
        print(f" - {msg}")
    if len(errors) > 5:
        print(f" ... {len(errors) - 5} more")
else:
    print(f"Using area {df['area'].iloc[0]} for {used_start} to {used_end}")
    df.head(10)

No per-plant data found for the requested areas/windows.
Tried:
 - DE_LU 2024-06-03 to 2024-06-04: No matching data found for Data item ACTUAL_GENERATION_OUTPUT_PER_UNIT_R3 [16.1.A] (10Y1001A1001A82H) and interval .
 - DE 2024-06-03 to 2024-06-04: No matching data found for Data item ACTUAL_GENERATION_OUTPUT_PER_UNIT_R3 [16.1.A] (10Y1001A1001A83F) and interval .
 - DE_LU 2024-01-15 to 2024-01-16: No matching data found for Data item ACTUAL_GENERATION_OUTPUT_PER_UNIT_R3 [16.1.A] (10Y1001A1001A82H) and interval .
 - DE 2024-01-15 to 2024-01-16: No matching data found for Data item ACTUAL_GENERATION_OUTPUT_PER_UNIT_R3 [16.1.A] (10Y1001A1001A83F) and interval .
 - DE_LU 2023-11-01 to 2023-11-02: No matching data found for Data item ACTUAL_GENERATION_OUTPUT_PER_UNIT_R3 [16.1.A] (10Y1001A1001A82H) and interval .
 ... 1 more


## Inspect coverage

In [7]:
print(f"Area: {df['area'].iloc[0]}")
print(f"Columns: {df.columns.tolist()}")
print(f"Unique units: {df['unit_name'].nunique()}")
print(f"Fuel types: {df['psr_type'].nunique()}")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")

fuel_counts = (
    df.groupby('psr_type')['unit_name']
    .nunique()
    .sort_values(ascending=False)
)
fuel_counts

TypeError: 'NoneType' object is not subscriptable

## Top generators by output in the window

In [ ]:
totals = (
    df.groupby(['unit_name', 'psr_type'])['value']
    .sum()
    .reset_index()
    .sort_values('value', ascending=False)
)
totals.head(15)

## Next steps

- If you get data for `DE_LU` or `DE`, extend the date range or filter by `psr_type` to focus on specific fuels.
- If you see no data, try a different period or check `docs/data-availability.md` in this repo for guidance on coverage by country and dataset.